# Merge da Base de Dados

Este notebook tem como objetivo realizar a unificação das diferentes tabelas relacionais.

Como o foco é a previsão do valor do frete, é necessário cruzar informações que originalmente estão dispersas (e.g. geolocalização do cliente e do vendedor). Esta etapa transforma o modelo relacional original em uma tabela única e estruturada.

Serão executadas as junções (merges) entre pedidos, itens, produtos, vendedores e clientes, apenas das colunas necessárias.

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
BASE_PATH = Path("../../../")
DATA_RAW = BASE_PATH / "data" / "raw"
DATA_INTERIM = BASE_PATH / "data" / "interim" / "T3" / "regressao"

orders = pd.read_csv(DATA_INTERIM / "orders_clean.csv")
order_items = pd.read_csv(DATA_RAW / "olist_order_items_dataset.csv")
products = pd.read_csv(DATA_INTERIM / "products_clean.csv")
customers = pd.read_csv(DATA_RAW / "olist_customers_dataset.csv")
sellers = pd.read_csv(DATA_RAW / "olist_sellers_dataset.csv")
geolocation = pd.read_csv(DATA_INTERIM / "geolocation_agg.csv")

**Fluxo de Junções:**

* **Items + Orders:** *Inner Join* entre itens e pedidos já filtrados (`orders_clean`).
* **+ Products:** *Inner Join* para adicionar peso e dimensões físicas dos produtos.
* **+ Customers & Sellers:** *Left Join* para vincular CEP e Estado (UF) de origem (vendedor) e destino (cliente).
* **Geolocation:** *Left Join* utilizando o CEP para adicionar latitude e longitude de ambas as pontas.

**Tratamento de Geolocalização Ausente:**
Alguns CEPs não possuem correspondência na base de coordenadas. Como a **distância** é uma feature obrigatória, removemos os registros resultantes que ficaram com latitude/longitude nulas (`dropna`).

In [3]:
# Items + Orders
df = order_items[['order_id', 'product_id', 'seller_id', 'price', 'freight_value']].merge(
    orders[['order_id', 'customer_id']], 
    on='order_id', 
    how='inner'
)

# Products
df = df.merge(
    products[['product_id', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']], 
    on='product_id', 
    how='inner'
)

# Customer
df = df.merge(
    customers[['customer_id', 'customer_zip_code_prefix', 'customer_state']], 
    on='customer_id', 
    how='left'
)

# Seller
df = df.merge(
    sellers[['seller_id', 'seller_zip_code_prefix', 'seller_state']], 
    on='seller_id', 
    how='left'
)

# Customer Geolocation
df = df.merge(
    geolocation[['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']]
       .rename(columns={'geolocation_lat': 'lat_customer', 'geolocation_lng': 'lng_customer'}),
    left_on='customer_zip_code_prefix',
    right_on='geolocation_zip_code_prefix',
    how='left'
)

# Seller Geolocation
df = df.merge(
    geolocation[['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']]
       .rename(columns={'geolocation_lat': 'lat_seller', 'geolocation_lng': 'lng_seller'}),
    left_on='seller_zip_code_prefix',
    right_on='geolocation_zip_code_prefix',
    how='left'
)

nulos_antes = df.isnull().sum()
df = df.dropna()
nulos_depois = df.isnull().sum()

display(pd.DataFrame({
    "Nulos antes": nulos_antes,
    "Nulos depois": nulos_depois
}))

,Nulos antes,Nulos depois
order_id,0,0
product_id,0,0
seller_id,0,0
price,0,0
freight_value,0,0
customer_id,0,0
product_weight_g,0,0
product_length_cm,0,0
product_height_cm,0,0
product_width_cm,0,0


## Exportação dos Dados

In [4]:
df.to_csv(DATA_INTERIM / "dataset_merged.csv", index=False)